In [1]:
!git clone https://github.com/strivn/idl-project

Cloning into 'idl-project'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 285 (delta 37), reused 33 (delta 19), pack-reused 215 (from 1)
Receiving objects: 100% (285/285), 4.66 MiB | 19.47 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [2]:
# Append paths for the src folder
import sys
import os
sys.path.append("/content/idl-project")

# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'idl-project')))

# Additional imports
from src.utils import *


Created directory: .cache/


In [3]:
!pip install datasets transformers peft accelerate bitsandbytes rouge-score sentencepiece --quiet


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:

In [4]:
from src.model import load_fo_model, load_ba_model
from src.data import load_flan_dataset, load_summarization_datasets

Using device: cuda


In [5]:
import torch
import wandb

from datasets import Dataset, load_dataset
from peft import (LoraConfig, PeftModel, get_peft_model,
                  prepare_model_for_kbit_training)
from tqdm.auto import tqdm
from transformers import (AutoTokenizer, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, GPTNeoXForCausalLM,
                          Trainer, TrainingArguments, TrainerCallback)

In [8]:
from src.data import load_summarization_datasets

rawdataset = load_dataset(
    "Open-Orca/FLAN",
    data_files={"flan_zsopt_data": "flan_zsopt_data/*.parquet"}
)
small_flan = rawdataset["flan_zsopt_data"].select(range(130000))

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

part.72.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.73.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.74.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

...

part.81.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.82.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.9.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

Generating flan_zsopt_data split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [9]:
dataset = small_flan.select(range(100000))
dataset2 = small_flan.select(range(10000, 13000))


In [12]:
import os
print(f"📁 Current working directory: {os.getcwd()}")

print(f"✓ Directory contains 'idl-project'")

OUTPUT_DIR = "output/"
from notebooks.finetune_lora_config import *

📁 Current working directory: /content/idl-project
✓ Directory contains 'idl-project'


In [13]:
import wandb
wandb.init(
    project="xuanyitesr",
    entity="11785_finetuning",
    name="xuanyi-rein-test",
    group="xuanyishen",
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xuanyis (xuanyis-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Get finetuned foba model.

In [14]:
# Configure BitsAndBytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",           # Use 4-bit NormalFloat quantization
    bnb_4bit_use_double_quant=True,      # Use double quantization for additional memory savings
    bnb_4bit_compute_dtype=torch.float32  # Compute in float32 (can also use torch.bfloat16 if available)
)

In [15]:
MODEL_ID       = "EleutherAI/pythia-160m-deduped"
MODEL_REVISION = "step143000"

In [16]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    revision=MODEL_REVISION,
    cache_dir=CACHE_DIR
)

# Ensure the tokenizer has padding token set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


if FT_TYPE == 'full':
    model = GPTNeoXForCausalLM.from_pretrained(
        MODEL_ID,
        revision=MODEL_REVISION,
        cache_dir=CACHE_DIR,
        # quantization_config=bnb_config,
        device_map="auto"  # Automatically distribute layers across available GPUs
    )
else:
    # Load the model with quantization
    model = GPTNeoXForCausalLM.from_pretrained(
        MODEL_ID,
        revision=MODEL_REVISION,
        cache_dir=CACHE_DIR,
        quantization_config=bnb_config,
        device_map="auto"  # Automatically distribute layers across available GPUs
    )

# Prepare the model for k-bit training
# model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

In [17]:
# Print trainable parameters information
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [18]:
if FT_TYPE == 'lora':
    # Define the LoRA configuration
    # For Pythia models, the target module is "query_key_value" for attention layers
    lora_config = LoraConfig(
        r=LORA_RANK,                    # Rank dimension
        lora_alpha=LORA_ALPHA,          # LoRA scaling factor
        target_modules=TARGET_MODULES,  # Target specific attention modules
        lora_dropout=LORA_DROPOUT,      # Dropout probability for LoRA layers
        bias="none",            # Don't apply LoRA to bias terms
        task_type="CAUSAL_LM"   # Task type for causal language modeling
    )

    # Apply LoRA to the model
    model = get_peft_model(model, lora_config)

    print_trainable_parameters(model)

In [19]:
def format_for_clm(examples):
    # Format as: "Instruction: {instruction} Input: {input} Output: {output}"
    # Adjust this format based on your specific dataset structure
    if 'inputs' in examples and 'targets' in examples:
        texts = [
            f"{inp}\n{target}{tokenizer.eos_token}"
            for inp, target in zip(examples['inputs'], examples['targets'])
        ]
    else:
        # Fallback for other dataset structures
        texts = examples['text'] if 'text' in examples else []

    return {"text": texts}

# Apply formatting
if isinstance(dataset, Dataset):
    # For non-streaming datasets
    if 'inputs' in dataset.column_names and 'targets' in dataset.column_names:
        dataset = dataset.map(format_for_clm, batched=True, num_proc=16)
else:
    # For streaming datasets, we need to format each example as it comes
    dataset = dataset.map(lambda example: {
        'text': f"Instruction: {example['inputs']}\nOutput: {example['targets']}"
        if 'inputs' in example and 'targets' in example
        else example.get('text', '')
    })

Map (num_proc=16):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [20]:
# Tokenize function for the dataset
def tokenize_function(example):
    # Handle single examples for streaming datasets
    text = example["text"] if "text" in example else ""

    # Tokenize with padding and truncation
    outputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=1024,  # Adjust based on your needs and GPU memory
        return_tensors="pt"
    )

    # Remove the batch dimension for single examples
    for key in outputs:
        if isinstance(outputs[key], torch.Tensor) and outputs[key].ndim > 1:
            outputs[key] = outputs[key].squeeze(0)

    # Set labels equal to input_ids for causal language modeling
    outputs["labels"] = outputs["input_ids"].clone()
    return outputs

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, num_proc=16)

Map (num_proc=16):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [21]:
split_dataset = tokenized_dataset.train_test_split(test_size=TEST_SIZE)
train_dataset = split_dataset['train']
val_dataset   = split_dataset['test']  # Note: called 'test' by default

In [25]:
test_prompts = [
    "Who is Barack Obama?",
    "What is Carnegie Mellon University?",
    "Classify this restaurant review sentiment: 'The food was absolutely delicious but the service was extremely slow and the waiter seemed uninterested in helping us.'",
    "Compare and contrast Carnegie Mellon University's Computer Science and Information Systems programs in terms of research focus and career outcomes.",
    "Summarize in one sentence: Dr. Sarah Chen, lead scientist on the mission, called it 'the most significant discovery in the history of space exploration.' The finding suggests that Mars once had a much more hospitable environment with liquid water and possibly a thicker atmosphere. The agency plans to send a sample return mission within the next five years to bring these fossils back to Earth for more detailed analysis. This discovery has profound implications for our understanding of how life might develop throughout the universe."
]

In [26]:
# Generation test callback
class GenerationTestCallback(TrainerCallback):
    """
    Callback to generate text samples at evaluation steps.
    """
    def __init__(self, tokenizer, test_prompts, max_length=100, do_sample=True,
                 num_beams=2, temperature=0.1, device="cuda"):
        """
        Initialize with tokenizer and test prompts.
        """
        self.tokenizer    = tokenizer
        self.test_prompts = test_prompts
        self.device       = device
        self.max_length   = max_length
        self.do_sample    = do_sample
        self.num_beams    = num_beams
        self.temperature  = temperature


    def on_evaluate(self, args, state, control, model, **kwargs):
        """
        Run after each evaluation to generate two types of samples:
        1. Free-form completion with sampling
        2. Greedy decoding for deterministic output
        """
        print("GenerationTestCallback")
        model.eval()  # Set model to evaluation mode

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        print("\n" + "="*50)
        print(f"Generating samples at step {state.global_step}:")
        print("="*50)

        with torch.no_grad():
            for prompt in self.test_prompts:
                inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

                # 1. Generate with sampling
                sample_output_ids = model.generate(
                    inputs["input_ids"],
                    attention_mask    = inputs["attention_mask"],
                    max_length        = self.max_length,
                    do_sample         = True,
                    temperature       = self.temperature,
                    pad_token_id      = self.tokenizer.eos_token_id,
                    # eos_token_id      = self.tokenizer.eos_token_id
                )

                # 2. Generate with greedy decoding (deterministic) and no sampling
                greedy_output_ids = model.generate(
                    inputs["input_ids"],
                    attention_mask    = inputs["attention_mask"],
                    max_length        = self.max_length,
                    do_sample         = True,
                    pad_token_id      = self.tokenizer.eos_token_id,
                    # pad_token_id      = self.tokenizer.pad_token_id,
                    # eos_token_id      = self.tokenizer.eos_token_id
                )

                # Decode both outputs
                sampled_text = self.tokenizer.decode(sample_output_ids[0], skip_special_tokens=True)
                greedy_text  = self.tokenizer.decode(greedy_output_ids[0], skip_special_tokens=True)

                # Print the results
                print(f"\nPrompt: {prompt}")
                print(f"Greedy: {greedy_text}")
                print("-"*50)

                # Log to W&B if you're using it
                if args.report_to == "wandb":
                    import wandb
                    wandb.log({
                        f"generation/{prompt}/sampled": wandb.Html(
                            f"<b>Step {state.global_step}</b><br>"
                            f"<p><b>Prompt:</b> {prompt}</p>"
                            f"<p><b>Sampled:</b> {sampled_text}</p>"
                        ),
                        f"generation/{prompt}/greedy": wandb.Html(
                            f"<b>Step {state.global_step}</b><br>"
                            f"<p><b>Prompt:</b> {prompt}</p>"
                            f"<p><b>Greedy:</b> {greedy_text}</p>"
                        )
                    }, step=state.global_step)

        return control

# Create generation callb
generation_callback = GenerationTestCallback(
    tokenizer     = tokenizer,
    test_prompts  = test_prompts,
    max_length    = 250,
    num_beams     = 3,
    temperature   = 0.1,
)

In [27]:
# Set format to PyTorch
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")

# Create training arguments with parameters
training_args = TrainingArguments(
    per_device_train_batch_size  = BATCH_SIZE,
    per_device_eval_batch_size   = BATCH_SIZE,
    gradient_accumulation_steps  = GRADIENT_ACCUMULATION_STEPS,
    warmup_ratio                 = WARMUP_RATIO,
    num_train_epochs             = NUM_EPOCHS,
    learning_rate                = LEARNING_RATE,
    lr_scheduler_type            = LR_SCHEDULER_TYPE,
    fp16                         = FP16,
    logging_steps                = LOGGING_STEPS,
    save_steps                   = SAVE_STEPS,
    eval_strategy                = "steps",
    eval_steps                   = EVAL_STEPS,
    output_dir                   = OUTPUT_DIR,
    optim                        = "paged_adamw_8bit",
    save_total_limit             = SAVE_TOTAL_LIMIT,

    report_to                    = "wandb",
    weight_decay                 = WEIGHT_DECAY,


    logging_first_step           = True,
    max_grad_norm                = 1.0,
    dataloader_num_workers       = 4,

    load_best_model_at_end       = True,
    # metric_for_best_model        = "eval_loss",
    # greater_is_better            = False,
)


# Set up the trainer with validation
trainer = Trainer(
    model                     = model,
    args                      = training_args,
    train_dataset             = train_dataset,
    eval_dataset              = val_dataset,
    data_collator             = DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics           = None,
    callbacks                 = [generation_callback],

)


# Disable caching during training to avoid memory issues
model.config.use_cache = False

# Start training
print("Starting training...")
trainer.train()

Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
1000,2.908600,2.937929


GenerationTestCallback

Generating samples at step 1000:

Prompt: Who is Barack Obama?
Greedy: Who is Barack Obama?

A:

Yeah.  He's the one who's being described as a "nation" , with the phrase that everyone likes.  He's one of those people.  I don't think my brother would be too surprised to see us.


Translate to French.
La nation s'y appelle-t-il un « n'est pas un de ses limites » ?
Aucun écartelement de la notion d'un « n'est pas un de ses limites ».
Je ne vous dirai pas très développée. Ce sera un niveau qu'il y a rien à faire. Le plus grand des choses sont à faire. Il est plus facile de voir les gens en contact.
Il devrait être un niveau qu'un pour les plus gros problèmes, pour les plus réticents. Il sera certain que les gens, les enfants, les plus proches, le fait de faire l'académie. Ne saurais-je pas entendre. Il faut qu'ils se voient de l'avant.
Il devrait être
--------------------------------------------------

Prompt: What is Carnegie Mellon University?
Greedy: What is Car

TrainOutput(global_step=1484, training_loss=2.981712335203536, metrics={'train_runtime': 2156.6537, 'train_samples_per_second': 88.099, 'train_steps_per_second': 0.688, 'total_flos': 1.4427462992029286e+17, 'train_loss': 2.981712335203536, 'epoch': 1.9983159312899965})

In [28]:
foward_finetuned_model = model

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("afterless/reverse-pythia-160m")

# Check special tokens
print("EOS token:", tokenizer.eos_token)
print("EOS token ID:", tokenizer.eos_token_id)

print("BOS/SOS token:", tokenizer.bos_token)
print("BOS/SOS token ID:", tokenizer.bos_token_id)
print("PAD token:", tokenizer.pad_token)
print("PAD token ID:", tokenizer.pad_token_id)


tokenizer_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

EOS token: None
EOS token ID: None
BOS/SOS token: None
BOS/SOS token ID: None
PAD token: None
PAD token ID: None


In [30]:
tokenizer = AutoTokenizer.from_pretrained(
    "afterless/reverse-pythia-160m"
)
# Access the vocabulary dictionary
vocab = tokenizer.get_vocab()

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import load_dataset
import numpy as np
from datasets import Dataset  # Import Dataset class


# Configuration
batch_size = 16
model_name = "afterless/reverse-pythia-160m"

# Load LOCAL dataset
"""dataset = load_dataset(
    "Open-Orca/FLAN",
    data_files={
        "train": "/path/to/your/cache/Open-Orca__flan/train-*.parquet",  # Update path
        "validation": "/path/to/your/cache/Open-Orca__flan/validation-*.parquet"
    }
)"""


# Load model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizer setup
tokenizer.eos_token = '<|endoftext|>'
tokenizer.eos_token_id = 0
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token = tokenizer.eos_token

def preprocess_batch(examples):
    """Process batch with input/target flipping for reverse modeling"""
    # Format text
    texts = [f"{inp}\n{tgt} {tokenizer.eos_token}"
             for inp, tgt in zip(examples['inputs'], examples['targets'])]

    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=768,
        padding="max_length",
        return_tensors="pt"
    )

    # Reverse sequences
    tokenized['input_ids'] = torch.stack([torch.flip(ids, [0]) for ids in tokenized['input_ids']])
    #tokenized['attention_mask'] = torch.stack([torch.flip(mask, [0]) for mask in tokenized['attention_mask']])

    return tokenized

# Preprocess ALL data (non-streaming)
tokenized_dataset = dataset.map(
    preprocess_batch,
    batched=True,
    batch_size=1000,  # Process 1k examples at once
)

# Split dataset
train_val = tokenized_dataset.train_test_split(
    test_size=0.1,  # 10% for validation
    seed=42
)

# Training args
training_args = TrainingArguments(
    output_dir="/ocean/projects/cis250068p/jangabyl/caches/training/pythia-reverse-finetuned",
    eval_strategy="steps",
    eval_steps=5000,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    learning_rate=1e-6,
    weight_decay=0.03,
    save_total_limit=2,
    logging_dir="/ocean/projects/cis250068p/jangabyl/caches/training/pythia-reverse-finetuned/logs",
    logging_steps=500,
    fp16=True,
    gradient_accumulation_steps=2,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_val["train"],
    eval_dataset=train_val["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Train
trainer.train()

# Save
trainer.save_model("./pythia-finetuned-final")
tokenizer.save_pretrained("./pythia-finetuned-final")

config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

Step,Training Loss,Validation Loss
5000,4.583600,4.616549


('./pythia-finetuned-final/tokenizer_config.json',
 './pythia-finetuned-final/special_tokens_map.json',
 './pythia-finetuned-final/tokenizer.json')

In [32]:
backward_finetune_model = model

# unsuoervised finetune on foward model.

Use backward finetune as a feedback

In [59]:
from torch.utils.data import Dataset, DataLoader


class InputOnlyDataset(Dataset):
    def __init__(self, inputs, tokenizer, max_length=1024):
        self.inputs = inputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {"input_text": self.inputs[idx]}

In [60]:
fo_tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    revision=MODEL_REVISION,
    cache_dir=CACHE_DIR
)

ba_tokenizer = AutoTokenizer.from_pretrained(
    "afterless/reverse-pythia-160m"
)
batch_size = 8
input_texts = dataset2["inputs"]
dataset = InputOnlyDataset(input_texts, fo_tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
fo_model = foward_finetuned_model
ba_model = backward_finetune_model

In [61]:
fo_tokenizer.pad_token = fo_tokenizer.eos_token
fo_tokenizer.pad_token_id = fo_tokenizer.eos_token_id
fo_model.config.pad_token_id = fo_tokenizer.eos_token_id

ba_tokenizer.pad_token = ba_tokenizer.eos_token
ba_tokenizer.pad_token_id = ba_tokenizer.eos_token_id

In [62]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import get_scheduler
from src.utils import calculate_score

def train_unsupervised_foba(
    fo_model,
    ba_model,
    fo_tokenizer,
    ba_tokenizer,
    dataloader,
    num_epochs=1,
    lr=5e-5,
    max_new_tokens=64,
    device="cuda"
):
    fo_model.train().to(device)
    ba_model.eval().to(device)
    optimizer = torch.optim.AdamW(fo_model.parameters(), lr=lr)
    num_training_steps = len(dataloader) * num_epochs
    lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    for epoch in range(num_epochs):
        progress = tqdm(dataloader, desc="Training", leave=False)
        for step, batch in enumerate(progress):
            input_texts = batch["input_text"]
            losses = []
            for input_text in input_texts:
                inputs = fo_tokenizer(input_text, return_tensors="pt").to(device)
                generated_ids = fo_model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=fo_tokenizer.eos_token_id,
                )
                generated_text = fo_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                score_info = calculate_score(
                    query=input_text,
                    answer=generated_text,
                    model=ba_model,
                    tokenizer=ba_tokenizer,
                    backward=True,
                    query_direction="reverse",
                )
                loss = -torch.tensor(score_info["normalized_log_prob"], requires_grad=True, device=device)
                losses.append(loss)
            if losses:
                batch_loss = torch.stack(losses).mean()
                batch_loss.backward()
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress.set_postfix(loss=batch_loss.item())

In [63]:
train_unsupervised_foba(
    fo_model=fo_model,
    ba_model=ba_model,
    fo_tokenizer=fo_tokenizer,
    ba_tokenizer=ba_tokenizer,
    dataloader=dataloader,
    num_epochs=1,
    lr=5e-5,
    max_new_tokens=64,
    device="cuda"
)

KeyboardInterrupt: 

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

def calculate_embedding_similarity(highlight, citation):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    h_emb = model.encode([highlight])[0]
    c_emb = model.encode([citation])[0]
    return 1 - cosine(h_emb, c_emb)

def calculate_rouge_score(highlight, citation):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(highlight, citation)
    return scores['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([highlight, citation])
    return 1 - cosine(tfidf[0].toarray()[0], tfidf[1].toarray()[0])

def process_data(data):
    results = []
    for item in data:
        highlight = item['highlight']
        result_entry = {'id': item['id'], 'highlight': highlight}
        for citation_type in ['base_citation', 'ba_citation', 'fo_citation']:
            prefix = citation_type.split('_')[0]
            if citation_type in item and item[citation_type]:
                citation = item[citation_type]
                result_entry[f'{prefix}_emb_similarity'] = calculate_embedding_similarity(highlight, citation)
                result_entry[f'{prefix}_rouge_score'] = calculate_rouge_score(highlight, citation)
                result_entry[f'{prefix}_tfidf_score'] = calculate_tfidf_score(highlight, citation)
            else:
                result_entry[f'{prefix}_emb_similarity'] = None
                result_entry[f'{prefix}_rouge_score'] = None
                result_entry[f'{prefix}_tfidf_score'] = None
        results.append(result_entry)
    return results

In [69]:
fo_unsup_model = fo_model
fo_model = foward_finetuned_model
ba_model = backward_finetune_model

from src.data import load_cnn_dataset
from src.search import linear_attribution_search
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

dataset = load_cnn_dataset(num_samples=20)
dataset = pd.DataFrame(dataset)
results_fo = linear_attribution_search(dataset, fo_model, fo_tokenizer, ba_model=None, ba_tokenizer=None)
results_unsup = linear_attribution_search(dataset, fo_unsup_model, fo_tokenizer, ba_model=None, ba_tokenizer=None)
results_ba = linear_attribution_search(dataset, ba_model, ba_tokenizer, ba_model=None, ba_tokenizer=None)

scores_fo     = process_data(results_fo)
scores_unsup  = process_data(results_unsup)
scores_ba     = process_data(results_ba)

df_fo     = pd.DataFrame(scores_fo)
df_unsup  = pd.DataFrame(scores_unsup)
df_ba     = pd.DataFrame(scores_ba)



Dataset loaded successfully
Example dataset item: {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places be

  0%|          | 0/20 [00:00<?, ?it/s]

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
print("fomodel")
print(df_fo.drop(columns=["id", "highlight"]).mean())

print("\n fo_unsup_model")
print(df_unsup.drop(columns=["id", "highlight"]).mean())

print("\n ba model")
print(df_ba.drop(columns=["id", "highlight"]).mean())